In [1]:
import math, json
import coiled, s3fs
import warnings
from collections import defaultdict
warnings.filterwarnings('ignore')
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import dask
import dask.array as da
import flox
import xarray as xr
xr.set_options(display_style='html')

import spei

import datetime, calendar


In [2]:
HIST_START = 1980
HIST_END = 2014

PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

NUM_BEST_MODELS = 3

In [3]:
FUTURE_START, FUTURE_END = 2080, 2099
FUTURE_SCENARIOS = ['ssp119', 'ssp126']

In [4]:
CITYLATLON = {}
with open('ghsl_500k.csv', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip() for i in line.split(',')]
        CITYLATLON['city_{0}'.format(items[0])] = (float(items[2]), float(items[3]), int(items[0]))

In [5]:
MODELS = {
    'tasmax': ('GFDL-ESM4', 'CanESM5', 'MRI-ESM2-0', 'IPSL-CM6A-LR', 'EC-Earth3-Veg-LR'),
    'tasmin': ('GFDL-ESM4', 'IPSL-CM6A-LR', 'CanESM5', 'MRI-ESM2-0', 'EC-Earth3-Veg-LR'),
    'pr': ('EC-Earth3-Veg-LR'), #'GFDL-ESM4', 'IPSL-CM6A-LR', 'CanESM5', 
    'hurs': ('GFDL-ESM4', 'CanESM5', 'MRI-ESM2-0', 'IPSL-CM6A-LR', 'EC-Earth3-Veg-LR'),
    'sfcWind': ('GFDL-ESM4', 'CanESM5', 'IPSL-CM6A-LR')
}

MODEL_URI = {}
with open('modelinfo.csv', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip() for i in line.split(',')]
        model, scenario, varname, the_uri = items
        MODEL_URI[(model, scenario, varname)] = the_uri

YEARLENGTH = {
    'GFDL-ESM4': 365,
    'CanESM5': 365,
    'MRI-ESM2-0': 366,
    'IPSL-CM6A-LR': 366,
    'EC-Earth3-Veg-LR': 366
}

In [6]:
MODEL_FAMILY = {'UKESM1-0-LL': 'HadAM',
 'NorESM2-MM': 'CCM',
 'NorESM2-LM': 'CCM',
 'MRI-ESM2-0': 'UCLA GCM',
 'MPI-ESM1-2-LR': 'ECMWF',
 'MPI-ESM1-2-HR': 'ECMWF',
 'MIROC6': 'MIROC',
 'MIROC-ES2L': 'MIROC',
 'KIOST-ESM': 'GFDL',
 'KACE-1-0-G': 'HadAM',
 'IPSL-CM6A-LR': 'IPSL',
 'INM-CM5-0': 'INM',
 'INM-CM4-8': 'INM',
 'HadGEM3-GC31-MM': 'HadAM',
 'HadGEM3-GC31-LL': 'HadAM',
 'GFDL-ESM4': 'GFDL',
 'GFDL-CM4_gr2': 'GFDL',
 'GFDL-CM4': 'GFDL',
 'FGOALS-g3': 'CCM',
 'EC-Earth3-Veg-LR': 'ECMWF',
 'EC-Earth3': 'ECMWF',
 'CanESM5': 'CanAM',
 'CNRM-ESM2-1': 'ECMWF',
 'CNRM-CM6-1': 'ECMWF',
 'CMCC-ESM2': 'CCM',
 'CMCC-CM2-SR5': 'CCM',
 'BCC-CSM2-MR': 'CCM',
 'ACCESS-ESM1-5': 'HadAM',
 'ACCESS-CM2': 'HadAM',
 'TaiESM1': 'CCM',
}

In [7]:
VARIABLES = {
    'tasmax': {
        'era_varname': 'maximum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'tasmin': {
        'era_varname': 'minimum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },

#    'pr': {
#        'era_varname': 'total_precipitation',
#        'nex_transform': lambda x: x * 86400,
#        'era_transform': lambda x: x * 1000
#    },
    'hurs': {
        'era_varname': None,
        'nex_transform': lambda x: x,
        'era_transform': lambda x: x
    },
#    'sfcWind': {
#        'era_varname': None,
#        'nex_transform': lambda x: x * 3600 / 1000,
#        'era_transform': lambda x: x * 3600 / 1000
#    },
}

In [8]:
CALIB_FXNS = {}
for varname in VARIABLES:
    CALIB_FXNS[varname] = {}
    for loc_id in [CITYLATLON[k][2] for k in list(CITYLATLON.keys())]:
        CALIB_FXNS[varname][int(loc_id)] = {}
for varname in VARIABLES:
    with open('bestmodels_{0}.txt'.format(varname), 'r') as ifile:
        for line in ifile.readlines():
            items = [i.strip() for i in line.split('\t')]
            varname, model, loc_id = items[1], items[2], int(items[0])
            CALIB_FXNS[varname][loc_id][model] = json.loads(items[3])

In [9]:
def returnperiod_value_daily(nex_varname, rp, latlon):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(PERCENTILE_STARTYEAR, PERCENTILE_ENDYEAR):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    d = np.sort(np.concatenate(allyears).flatten())
    d = d[d > 0.01]  # Only consider actual positive events
    vals, counts = np.unique(d, return_counts=True)
    freqs = counts / d.size
    cdf_y = np.cumsum(freqs)
    targetfreq = (PERCENTILE_ENDYEAR - PERCENTILE_STARTYEAR + 1) / rp
    return np.interp(1-targetfreq, vals, cdf_y)
    

def calendardate_percentiles(nex_varname, q, latlon, sh_hem=False):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(PERCENTILE_STARTYEAR, PERCENTILE_ENDYEAR):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    if not sh_hem:
        return np.percentile(np.vstack(allyears), q, axis=0)
    else:
        res = np.percentile(np.vstack(allyears), q, axis=0)
        return np.concatenate([res[152:], res[:152]])

def wholeyear_percentile(nex_varname, q, latlon):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    return np.percentile(np.concatenate(allyears).flatten(), q)

def yearextreme_percentile(nex_varname, q, latlon, wantmax):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append([np.min, np.max][int(wantmax)](VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False))))
    return np.percentile(np.array(allyears), q)

def thresholdexceedance_mediancount(nex_varname, threshold, latlon, want_gte):
    era_varname = VARIABLES[nex_varname]['era_varname']
    data = VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=PERCENTILE_STARTYEAR, end_year=PERCENTILE_ENDYEAR, southern_hem=False))
    if data.size % 365 != 0:
        raise Exception('Data array length is not an integer multiple of 365')
    byyear = data.reshape(data.size//365, 365)
    if want_gte:
        return np.median(np.sum(byyear >= threshold, axis=1))
    else:
        return np.median(np.sum(byyear <= threshold, axis=1))
    
def max_sevendaymean(arr):
    idx_a = 0
    idx_b = min(7, arr.size-1)
    allmeans = []
    while idx_b <= arr.size:
        allmeans.append(np.mean(arr[idx_a:idx_b]))
    return max(allmeans)
def wholeyear_percentile(nex_varname, q, latlon):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    return np.percentile(np.concatenate(allyears).flatten(), q)

def get_rmsd(d1, d2):
    c1 = seasonal_means(d1)
    c2 = seasonal_means(d2)
    return np.sqrt(np.mean(np.sum((c1 - c2)**2)))

def count_runs(tf_array, min_runsize):
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    count = 0
    for idx in range(starts.size):
        if ends[idx] - starts[idx] >= min_runsize:
            count += 1
    return count

def removeLeapDays(arr, start_year, end_year, southern_hem):
    if not southern_hem:
        indices = []
        jan1_idx = 0
        for year in range(start_year, end_year+1):
            indices += [jan1_idx + i for i in range(365)]
            jan1_idx += 365
            if calendar.isleap(year):
                jan1_idx += 1
        return arr[indices]
    else:
        indices = []
        jul1_idx = 0
        for year in range(start_year-1, end_year):
            indices += [jul1_idx + i for i in range(365)]
            jul1_idx += 365
            if calendar.isleap(year):
                jul1_idx += 1
        return arr[indices]


def get_var(varname, model, latlon, start_year, end_year, sh_year, scenario):
    dataset = DATASETS[(varname, model, scenario)]
    if not sh_year:
        dates = ('{0}-01-01'.format(start_year), '{0}-12-31'.format(end_year))
    else:
        dates = ('{0}-07-01'.format(start_year-1), '{0}-06-30'.format(end_year))
    
    ds = dataset.sel(time=slice(*dates)).sel(lat=latlon[0], lon=latlon[1], method='nearest')
    ds = ds.values
    if YEARLENGTH[model] == 366:
        return removeLeapDays(ds, start_year, end_year, sh_year)
    else:
        return ds

def quarters(d, start_year, end_year, sh_year=False):
    q2 = []  # 60-151
    q3 = []  # 152-243
    q4 = []  # 244-334
    q1 = []  # 335-59
    if not sh_year:
        jan1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
            q1.append(tmp)
            q2.append(d[jan1_idx + 60 : jan1_idx + 152])
            q3.append(d[jan1_idx + 152 : jan1_idx + 244])
            q4.append(d[jan1_idx + 244 : jan1_idx + 335])

            jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q2)
        jja_res = np.vstack(q3)
        son_res = np.vstack(q4)
        djf_res = np.vstack(q1)
    else:
        jul1_idx = 365
        for year in range(start_year, end_year+1):
            tmp = np.concatenate((d[jul1_idx - 365 : jul1_idx - 365 + 60], d[jul1_idx + 335 : jul1_idx + 365]), axis=0)
            q3.append(tmp)
            q4.append(d[jul1_idx + 60 : jul1_idx + 152])
            q1.append(d[jul1_idx + 152 : jul1_idx + 244])
            q2.append(d[jul1_idx + 244 : jul1_idx + 335])

            jul1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q4)
        jja_res = np.vstack(q1)
        son_res = np.vstack(q2)
        djf_res = np.vstack(q3)
    return mam_res, jja_res, son_res, djf_res
    
def seasonal_means(d):
    q = quarters(d, HIST_START, HIST_END)
    return np.array([np.mean(q[0], axis=1), np.mean(q[1], axis=1), np.mean(q[2], axis=1), np.mean(q[3], axis=1)])

def calibration_function(hist_obs, hist_mod):
# Calibration functions are P-P plots of historical and modeled values

    source = np.sort(hist_obs.flatten())
    target= np.sort(hist_mod.flatten())
   
    if (np.max(source) == 0 and np.min(source) == 0):
        return np.arange(0, target.size) / target.size
    if (np.max(target) == 0 and np.min(target) == 0):
        return np.arange(0, source.size) / source.size
    new_indices = []

    for target_idx, target_value in enumerate(target):
        if target_idx < len(source):
            source_value = source[target_idx]
            if source_value > target[-1]:
                new_indices.append(target.size - 1)
            else:
                new_indices.append(np.argmax(target >= source_value))
    return np.array(new_indices) / source.size

def calibrate_component(uncalibrated_data, calibration_fxn):
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

def get_gamma(count, size):
    return np.random.gamma(shape = count + 0.5, size=size)
def get_beta(count, num, size):
    return np.random.beta(a = count + 0.5, b = num - count + 0.5, size=size)

In [10]:
class FWICLASS:
        # https://d1ied5g1xfgpx8.cloudfront.net/pdfs/36461.pdf
    def __init__(self,temp,rhum,wind,prcp):
        self.h = rhum
        self.t = temp
        self.w = wind
        self.p = prcp
    def FFMCcalc(self,ffmc0):
        mo = (147.2*(101.0 - ffmc0))/(59.5 + ffmc0) #*Eq. 1*#
        if (self.p > 0.5):
            rf = self.p - 0.5 #*Eq. 2*#
            if(mo > 150.0):
                mo = (mo+42.5*rf*math.exp(-100.0/(251.0-mo))*
                (1.0 - math.exp(-6.93/rf))) + (.0015*(mo - 150.0)**2)*math.sqrt(rf) #*Eq. 3b*#
            elif mo <= 150.0:
                mo = mo+42.5*rf*math.exp(-100.0/(251.0-mo))*(1.0 - math.exp(-6.93/rf)) #*Eq. 3a*#
            if(mo > 250.0):
                mo = 250.0
        ed = .942*(self.h**.679) + (11.0*math.exp((self.h-100.0)/10.0))+0.18*(21.1-self.t) *(1.0 - 1.0/math.exp(.1150 * self.h)) #*Eq. 4*#
        if(mo < ed):
            ew = .618*(self.h**.753) + (10.0*math.exp((self.h-100.0)/10.0)) + .18*(21.1-self.t)*(1.0 - 1.0/math.exp(.115 * self.h)) #*Eq. 5*#
            if(mo <= ew):
                kl = .424*(1.0-((100.0-self.h)/100.0)**1.7)+(.0694*math.sqrt(self.w)) *(1.0 - ((100.0 - self.h)/100.0)**8) #*Eq. 7a*#
                kw = kl * (.581 * math.exp(.0365 * self.t)) #*Eq. 7b*#
                m = ew - (ew - mo)/10.0**kw #*Eq. 9*#
            elif mo > ew:
                m = mo
        elif(mo == ed):
            m = mo
        elif mo > ed:
            kl =.424*(1.0-(self.h/100.0)**1.7)+(.0694*math.sqrt(self.w))* (1.0-(self.h/100.0)**8) #*Eq. 6a*#
            kw = kl * (.581*math.exp(.0365*self.t)) #*Eq. 6b*#
            m = ed + (mo-ed)/10.0 ** kw #*Eq. 8*#
        ffmc = (59.5 * (250.0 -m)) / (147.2 + m)  #*Eq. 10*#
        if (ffmc > 101.0):
            ffmc = 101.0
        if (ffmc <= 0.0):
            ffmc = 0.0
        return ffmc

    def DMCcalc(self,dmc0,mth):
        el = [6.5,7.5,9.0,12.8,13.9,13.9,12.4,10.9,9.4,8.0,7.0,6.0]
        t = self.t
        if (t < -1.1):
            t = -1.1
        rk = 1.894*(t+1.1) * (100.0-self.h) * (el[mth-1]*0.0001) #*Eqs. 16 and 17*#
        if self.p > 1.5:
            ra= self.p
            rw = 0.92*ra - 1.27  #*Eq. 11*#
            wmi = 20.0 + 280.0/math.exp(0.023*dmc0)  #*Eq. 12*#
            if dmc0 <= 33.0:
                b = 100.0 /(0.5 + 0.3*dmc0)  #*Eq. 13a*#
            elif dmc0 > 33.0:
                if dmc0 <= 65.0:
                    b = 14.0 - 1.3*math.log(dmc0)  #*Eq. 13b*#
                elif dmc0 > 65.0:
                    b = 6.2 * math.log(dmc0) - 17.2  #*Eq. 13c*#
            wmr = wmi + (1000*rw) / (48.77+b*rw)  #*Eq. 14*#
            pr = 43.43 * (5.6348 - math.log(wmr-20.0))  #*Eq. 15*#
        elif self.p <= 1.5:
            pr = dmc0
        if (pr<0.0):
            pr = 0.0
        dmc = pr + rk
        if(dmc<= 1.0):
            dmc = 1.0
        return dmc

    def DCcalc(self,dc0,mth):
        fl = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
        t = self.t
        if(t < -2.8):
            t = -2.8
        pe = (0.36*(t+2.8) + fl[mth-1] )/2  #*Eq. 22*#
        if pe <=0.0:
            pe = 0.0
        if (self.p > 2.8):
            ra = self.p
            rw = 0.83*ra - 1.27 #*Eq. 18*#
            smi = 800.0 * math.exp(-dc0/400.0) #*Eq. 19*#
            dr = dc0 - 400.0*math.log( 1.0+((3.937*rw)/smi) ) #*Eqs. 20 and 21*#
            if (dr > 0.0):
                dc = dr + pe
            else:
                dc = pe
        elif self.p <= 2.8:
            dc = dc0 + pe
        return dc

    def ISIcalc(self,ffmc):
        mo = 147.2*(101.0-ffmc) / (59.5+ffmc)  #*Eq. 1*#
        ff = 19.115*math.exp(mo*-0.1386) * (1.0+(mo**5.31)/49300000.0)  #*Eq. 25*#
        isi = ff * math.exp(0.05039*self.w)  #*Eq. 26*#
        return isi

    def BUIcalc(self,dmc,dc):
        if dmc <= 0.4*dc:
            bui = (0.8*dc*dmc) / (dmc+0.4*dc)
        else:
            bui = dmc-(1.0-0.8*dc/(dmc+0.4*dc))*(0.92+(0.0114*dmc)**1.7)
        if bui <0.0:
            bui = 0.0
        return bui

    def FWIcalc(self,isi,bui):
        if bui <= 80.0:
            bb = 0.1 * isi * (0.626*bui**0.809 + 2.0)  #*Eq. 28a*#
        else:
            bb = 0.1*isi*(1000.0/(25. + 108.64/math.exp(0.023*bui)))  #*Eq. 28b*#
        if(bb <= 1.0):
            fwi = bb  #*Eq. 30b*#
        else:
            fwi = math.exp(2.72 * (0.434*math.log(bb))**0.647)  #*Eq. 30a*#
        return fwi
#End of class FWI Class
def getFWI(mth,day,temp,rhum,wind,prcp):
    ffmc0 = 85.0
    dmc0 = 6.0
    dc0 = 15.0
    rhum = np.minimum(rhum, 100.0)
    fwisystem = FWICLASS(temp,rhum,wind,prcp)
    ffmc = fwisystem.FFMCcalc(ffmc0)
    dmc = fwisystem.DMCcalc(dmc0,mth)
    dc = fwisystem.DCcalc(dc0,mth)
    isi = fwisystem.ISIcalc(ffmc)
    bui = fwisystem.BUIcalc(dmc,dc)
    fwi = fwisystem.FWIcalc(isi,bui)
    ffmc0 = ffmc
    dmc0 = dmc
    dc0 = dc

    #return ffmc,dmc,dc,isi,bui,fwi
    return fwi

In [11]:
class Hazard:
    pass

class FireWeather(Hazard):
    def __init__(self, threshold):
        self.varname = 'tasmax+pr+hurs+sfcWind'
        self.probmodel = 'binomial'
        self.threshold = threshold
        
    def count_nc(self, datalist, targetcount):
        data_temp = datalist[0]
        data_prcp = datalist[1]
        data_rhum = datalist[2]
        data_wind = datalist[3]
        data_month = [datetime.date.fromordinal(i+1).month for i in range(365)] * (data_temp.size//365)
        data_day = [datetime.date.fromordinal(i+1).day for i in range(365)] * (data_temp.size//365)
        data = np.array([getFWI(data_month[idx], data_day[idx], data_temp[idx], data_rhum[idx], data_wind[idx], data_prcp[idx]) for idx in range(data_temp.size)])
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        return np.sum(np.abs(np.sum(byyear >= self.threshold, axis=1) - targetcount) < 0.5)

class Tempwave_simple(Hazard):
    def __init__(self, varname, min_duration, threshold, want_gte=True):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.varname = varname
        self.want_gte = want_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data = datalist[0]
        if type(self.threshold) in (float, int, np.float64, np.int32):
            threshold = self.threshold
        else:   # type is np array
            threshold = np.array([])
            while threshold.size < data.size:
                threshold = np.concatenate([threshold, self.threshold])
        if self.want_gte:
            tf_array = data >= threshold
        else:
            tf_array = data <= threshold
        return count_runs(tf_array, self.min_duration)

def wetbulbtemp(T, RH):
# JA Knox et al. 2017. Two simple and accurate approximations for wet-bulb
# temperature in moist conditions, with forecasting applications. Bull. Am.
# Meteorol. Soc. 98(9): 1897-1906. doi:10.1175/BAMS-D-16-0246.1
    T = T.astype(np.float64)
    rh_percent = RH.astype(np.float64)
    return T * np.arctan(0.151977 * np.sqrt(rh_percent + 8.313659)) + np.arctan(T + rh_percent) - np.arctan(rh_percent - 1.676331) + ((0.00391838 * ((rh_percent)**(3/2))) * np.arctan(0.023101 * rh_percent)) - 4.686035


def wetbulbtemp_chen(T, RH):
    return -4391976 + (0.0198197 * RH) + (0.526359 * T) + (0.00730271 * RH * T) + (0.00024315 * RH * RH) - (0.0000258101 * T * RH * RH)

class WetbulbHeatwave(Hazard):
    def __init__(self, wbgt_threshold, min_duration):
        self.varname = 'tasmax+hurs'
        self.min_duration = min_duration
        self.wbgt_threshold = wbgt_threshold
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        tf_array = data >= self.wbgt_threshold
        return count_runs(tf_array, self.min_duration)

class WetbulbDays(Hazard):
    def __init__(self, wbgt_threshold):
        self.varname = 'tasmax+hurs'
        self.wbgt_threshold = wbgt_threshold
        self.probmodel = 'binomial'
    def count(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        byyear = data.reshape(data.size//365, 365)
        return np.sum((np.max(byyear, axis=1) >= self.wbgt_threshold) * 1)
    
    def count_nc(self, datalist, targetcount):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        return np.sum(np.abs(np.sum(byyear >= self.wbgt_threshold, axis=1) - targetcount) < 0.5)
    
class Heatwave_highlow(Hazard):
    def __init__(self, hightemp, lowtemp, min_duration):
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return count_runs(tf_array_tx * tf_array_tn, self.min_duration)

class Drought_SPI(Hazard):
    def __init__(self, min_duration):
        self.varname = 'pr'
        self.probmodel = 'Poisson'
        self.min_duration = min_duration
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        
        t=pd.date_range(start='1980-01-01', end='{0}-12-31'.format(1980 + (data.size//365) - 1), freq='D')
        t = t[~((t.month == 2) & (t.day == 29))]
        
        tries = 0
        success = False
        while (not success) and tries < 5:
            try:
                droughtdays = spei.spi(pd.Series(data, index=t)).to_numpy()
                success = True
            except:
                tries += 1
        if success:
            return count_runs(droughtdays < -2, self.min_duration)
        else:
            print("Returning -9999")
            return -9999
    def count_nc(self, datalist, targetcount):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        
        t=pd.date_range(start='1980-01-01', end='{0}-12-31'.format(1980 + (data.size//365) - 1), freq='D')
        t = t[~((t.month == 2) & (t.day == 29))]
        
        droughtdays = spei.spi(pd.Series(data, index=t)).to_numpy()
        byyear = droughtdays.reshape(data.size//365, 365)
        
        return np.sum(np.abs(np.sum(byyear <= -2, axis=1) - targetcount) < 0.5)
    
    
class Threshold_simple(Hazard):
    def __init__(self, varname, var_threshold, want_gte):
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_gte = want_gte
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        if self.want_gte:
            return np.sum(np.sum(byyear >= self.var_threshold, axis=1) >= self.count_threshold)
        else:
            return np.sum(np.sum(byyear <= self.var_threshold, axis=1) >= self.count_threshold)
    def count_nc(self, datalist, targetcount):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        return np.sum(np.abs(np.sum(byyear >= self.var_threshold, axis=1) - targetcount) < 0.5)
        

class Hotdays_inrange(Hazard):
    def __init__(self, hightemp, lowtemp):
        self.varname = 'tasmax'
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        tf_array_high = data <= self.hightemp
        tf_array_low = data >= self.lowtemp
        return runs(tf_array_high * tf_array_low, self.min_duration, 'count')

In [10]:
class Hazard:
    def get_estimates(self, latlon, start_year, end_year, datasets, calib_fxns):
        sh_year = int(latlon[0] < 0)
        fut_mod = {}
        varnames = self.varname.split('+')
        for varname in varnames:
            for model in calib_fxns[varname].keys():
                ds = datasets[varname][model]

                if YEARLENGTH[model] == 366:
                    ds = removeLeapDays(ds, start_year, end_year, sh_year)
                fut_mod[(varname, model)] = ds
        best_models = []
        for idx in range(NUM_BEST_MODELS):
            best_models.append('+'.join([list(calib_fxns[varname].keys())[idx] for varname in varnames]))
        res_sum = {modelplus: [0, 0, 0] for modelplus in best_models}
        for targetval in self.targetvals:
            for modelplus in best_models:
                calib_data = []
                for idx, varname in enumerate(varnames):
                    model = modelplus.split('+')[idx]
                    calib_data.append(np.array(calibrate(fut_mod[(varname, model)], calib_fxns[varname][model])))
                count = self.val_nc([cd[[0,152][int(sh_year)]:[len(cd),-213][int(sh_year)]] for cd in calib_data], targetval)
                posterior_rateparams = get_beta(count, end_year - start_year + 1, 10000)
                if count == 0:
                    posterior_rateparams = np.zeros(10000)
                posterior_draws = np.random.binomial(end_year - start_year + 1, posterior_rateparams, 10000)
                probs = [np.percentile(posterior_draws, q) / (end_year - start_year + 1) for q in (25, 50, 75)]
                res_sum[modelplus] = [res_sum[modelplus][idx] + (targetval * probs[idx] ) for idx in (0, 1, 2)]
        return {modelplus: [max(min(res_sum[modelplus][idx], max(self.targetvals)), 0) for idx in (0, 1, 2)] for modelplus in res_sum}

class ThresholdDays(Hazard):
    def __init__(self, hazname, varname, var_threshold, want_max, targetvals):
        self.hazname = hazname
        self.varname = varname
        self.var_threshold = var_threshold
        self.targetvals = targetvals
        self.want_max = want_max
        self.probmodel = 'binomial'

    def val_nc(self, datalist, targetval):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        if self.want_max:
            return np.sum(np.sum(byyear <= self.var_threshold, axis=1) == targetval)
        else:
            return np.sum(np.sum(byyear <= self.var_threshold, axis=1) == targetval)
    
class WetbulbDays(Hazard):
    def wetbulbtemp(T, RH):
    # JA Knox et al. 2017. Two simple and accurate approximations for wet-bulb
    # temperature in moist conditions, with forecasting applications. Bull. Am.
    # Meteorol. Soc. 98(9): 1897-1906. doi:10.1175/BAMS-D-16-0246.1
        T = T.astype(np.float64)
        rh_percent = RH.astype(np.float64)
        return T * np.arctan(0.151977 * np.sqrt(rh_percent + 8.313659)) + np.arctan(T + rh_percent) - np.arctan(rh_percent - 1.676331) + ((0.00391838 * ((rh_percent)**(3/2))) * np.arctan(0.023101 * rh_percent)) - 4.686035

    def __init__(self, hazname, wbgt_threshold, targetvals):
        self.hazname = hazname
        self.varname = 'tasmax+hurs'
        self.wbgt_threshold = wbgt_threshold
        self.probmodel = 'binomial'
        self.targetvals = targetvals

    def val_nc(self, datalist, targetcount):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        return np.sum(np.sum(byyear >= self.wbgt_threshold, axis=1) == targetcount)

class DroughtSPIDays(Hazard):
    def __init__(self, hazname, targetvals):
        self.hazname = hazname
        self.varname = 'pr'
        self.probmodel = 'Poisson'
        self.targetvals = targetvals

    def val_nc(self, datalist, targetcount):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        
        t=pd.date_range(start='1980-01-01', end='{0}-12-31'.format(1980 + (data.size//365) - 1), freq='D')
        t = t[~((t.month == 2) & (t.day == 29))]
        
        droughtdays = spei.spi(pd.Series(data, index=t)).to_numpy()
        byyear = droughtdays.reshape(data.size // 365, 365)
        
        return np.sum(np.sum(byyear <= -2, axis=1) == targetcount)
    
class ExtremestVal(Hazard):
    def __init__(self, hazname, varname, want_max, targetvals):
        self.hazname = hazname
        self.varname = varname
        self.want_max = want_max
        self.probmodel = 'binomial'
        self.targetvals = targetvals
        
    def val_nc(self, datalist, targetval):
        data = datalist[0]
        byyear = data.reshape(data.size // 365, 365)
        if self.want_max:
            return np.sum(np.round(np.max(byyear, axis=1)) == targetval)
        else:
            return np.sum(np.round(np.min(byyear, axis=1)) == targetval)
    
class RangeDays(Hazard):
    def __init__(self, hazname, varname, low_threshold, high_threshold, targetvals):
        self.hazname = hazname
        self.varname = varname
        self.low_threshold = low_threshold
        self.high_threshold = high_threshold
        self.probmodel = 'binomial'
        self.targetvals = targetvals
        
    def val_nc(self, datalist, targetcount):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size // 365, 365)
        return np.sum(np.sum((byyear >= self.low_threshold)*(byyear <= self.high_threshold), axis=1) == targetcount)
    
class RangeDaysTwovar(Hazard):
    def __init__(self, hazname, varname, low_threshold, high_threshold, targetvals):
        self.hazname = hazname
        self.varname = varname
        self.low_threshold = low_threshold
        self.high_threshold = high_threshold
        self.probmodel = 'binomial'
        self.targetvals = targetvals
        
    def val_nc(self, datalist, targetcount):
        datalow = datalist[0]
        datahigh = datalist[1]
        datamid = (datalist[0] + datalist[1]) / 2
        if datamid.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = datamid.reshape(datamid.size // 365, 365)
        return np.sum(np.sum((byyear >= self.low_threshold)*(byyear <= self.high_threshold), axis=1) == targetcount)
            
            

In [11]:
MODEL_URI = {}
with open('modelinfo.csv', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip() for i in line.split(',')]
        model, scenario, varname, the_uri = items
        MODEL_URI[(model, scenario, varname)] = the_uri
def get_futmods_everywhere(varname, scenario, model, southern_hem):
    def uri(model, scenario, varname):
        return MODEL_URI[(model, scenario, varname)]
    def s3open_cmip(path):
        fs = s3fs.S3FileSystem(anon=True)
        return s3fs.S3Map(path, s3=fs)
    thefile = s3open_cmip(uri(model, scenario, varname))
    ds = xr.open_mfdataset([thefile], engine='zarr', parallel=True)
    if southern_hem:
        ds = ds[varname].resample(time='D').sum().sel(time=slice('{0}-07-01'.format(FUTURE_START-1), '{0}-06-30'.format(FUTURE_END)))
    else:
        ds = ds[varname].resample(time='D').sum().sel(time=slice('{0}-01-01'.format(FUTURE_START), '{0}-12-31'.format(FUTURE_END)))
    return VARIABLES[varname]['nex_transform'](ds).chunk({"time": -1, "lat": "auto", "lon": "auto"})

In [24]:
cluster.shutdown()

In [ ]:
cluster = coiled.Cluster(n_workers=50, name='cluster_1', compute_purchase_option="spot_with_fallback", shutdown_on_close=False)
client = cluster.get_client()

Output()

In [19]:
cluster.shutdown()

In [17]:
pr_rp100 = {}
with open('precip_rp100.txt', 'r') as ifile:
    lines = ifile.readlines()
    for line in lines:
        items = line.split('\t')
        loc_id, val = int(items[0]), float(items[1])
        pr_rp100[loc_id] = val

In [18]:
def getdata(varnames, scenario, lat, lon):
    return {varname: {model: get_futmods_everywhere(varname, scenario, model, lat < 0).sel(lat=lat, lon=lon, method='nearest').values for model in MODELS[varname]} for varname in varnames}

def do_locationhazard(hazard, loc_id, latlon, scenario, calib_fxns):
    lat, lon = latlon
    varnames = hazard.varname.split('+')
    datasets = getdata(varnames, scenario, lat, lon)
    return loc_id, lat, lon, hazard.hazname, scenario, '{0}-{1}'.format(FUTURE_START, FUTURE_END), hazard.get_estimates(latlon, FUTURE_START, FUTURE_END, datasets, calib_fxns)

In [23]:
%%time
futures = []
for cityname in list(CITYLATLON.keys())[:100]:
    lat, lon, loc_id = CITYLATLON[cityname]
    pr_rp100_val = pr_rp100[loc_id]
    HAZARDS = [
        RangeDaysTwovar('numdays_tmax_betw_23.9_34', 'tasmin+tasmax', 23.9, 34, range(0, 366)),
        ExtremestVal('max_tmax', 'tasmax', True, range(15, 60)),
        #DroughtSPIDays('numdays_SPI_lte_-2', range(0, 366)),
        WetbulbDays('numdays_Twb_gte_31', 31, range(0, 366)),
        #ThresholdDays('numdays_pr_gte_rp100', 'pr', pr_rp100_val, True, range(0, 200))
    ]
    for hazard in HAZARDS:
        for scenario in FUTURE_SCENARIOS:
            varnames = hazard.varname.split('+')
            calib_fxns = {varname: CALIB_FXNS[varname][loc_id] for varname in varnames}
            futures.append(client.submit(do_locationhazard, hazard, loc_id, (lat, lon), scenario, calib_fxns))


KeyboardInterrupt: 

In [30]:
%%time
for future in futures:
    if future.status=='finished':
        result = future.result()
        with open('tempinds_996.csv', 'a') as ofile:
            loc_id, lat, lon, hazard.hazname, scenario, year_range, valdict = result
            for model in valdict:
                ofile.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9}\n'.format(loc_id, lat, lon, hazard.hazname, scenario, model, year_range, valdict[model][0], valdict[model][1], valdict[model][2]))

CPU times: total: 406 ms
Wall time: 5.88 s


In [34]:
for quarter in range(4):
    obs_10 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 10)
    obs_90 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 90)
    for model in best_models_tx:
        mod = quarters(hist_mods_tx[model] - 273.15, HIST_START, HIST_END)[quarter].flatten()
        print('{0}: min modeled value does not exceed observed 10th percentile  {1}'.format(model, min(mod) <= obs_10))
        print('{0}: max modeled value does not exceed observed 90th percentile  {1}'.format(model, max(mod) >= obs_90))


GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled